In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from env import user, host, password
from wrangle import get_connection, new_zillow_data, get_zillow_data

# Acquire

In [ ]:
df = get_zillow_data()
df.head()

In [ ]:
# Lets get shorter and easier to understand names
df = df.rename(columns= {'bedroomcnt' : 'bedrooms', 
                         'bathroomcnt' : 'bathrooms', 
                         'calculatedfinishedsquarefeet' : 'area_sqft', 
                         'taxvaluedollarcnt' : 'tax_value', 
                         'yearbuilt' : 'year_built', 
                         'taxamount' : 'tax_amount'})

In [ ]:
# Checking the renamed columns
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

# Get a boolean telling you if there are nulls in a column
# df.isnull().any()

# Get list of column names that have nulls
# df.columns[df.isnull().any()]

In [ ]:
df.hist()

In [ ]:
# Make a nicer histogram than the previous histplot
plt.figure(figsize=(16, 3))

# List of columns
#cols = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 
#       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips']
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):
    
    # Plot number should start at 1, but i starts at 0
    plot_number = i + 1
    
    # Create subplots
    plt.subplot(1, len(cols), plot_number)
    
    # Add a title to the subplots
    plt.title(col)
    
    # Make a histogram for each column
    df[col].hist(bins=5)
    # df.col.hist(bins=5)   for some reason this alternative line gets an error
    
    # Hide the grids in the subplots
    plt.grid(False)
    
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
sns.boxplot(data=df.drop(columns='fips'))

In [ ]:
# Make a nicer boxplot than the previous boxplot
plt.figure(figsize=(16, 3))

# List of columns
#cols = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 
#       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips']

#cols = [col for col in df.columns if col not in ['fips', 'year_built']]
cols = df.columns

for i, col in enumerate(cols):
    
    # Plot number should start at 1, but i starts at 0
    plot_number = i + 1
    
    # Create subplots
    plt.subplot(1, len(cols), plot_number)
    
    # Add a title to the subplots
    plt.title(col)
    
    # Make a histogram for each column
    sns.boxplot(data = df[[col]])
    
    # Hide the grids in the subplots
    plt.grid(False)
    
#plt.show()
plt.tight_layout()
# Some of my plots are shaped differently than John's, 
# perhaps because of my method of acquiring?

### Takeaways for Acquire <br>
- All columns are float64 <br>
- Only taxamount and bathroomcnt need to be float, the rest can be int <br>
- Lots of null values in most columns <br>
- Some columns have a huge range of values <br>
- Some columns have extreme outliers <br>
- Some columns do not have a normal distribution

# Prepare

In [ ]:
# Check for nulls again
df.isnull().sum()
# I will impute the null values later

In [ ]:
# Nulls are rare for bedrooms and bathrooms
# Nulls may not appear in train after splitting
# Thus, I am dropping nulls for bedroom and bathroom now
df.bedrooms.dropna()
df.bathrooms.dropna()
df.isnull().sum()

In [ ]:
# Replace a whitespace sequence or empty with a NaN value 
# and reassign this manipulation to df.
df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
def remove_outliers(df, k, col_list):
    '''
    Remove outliers from a list of columns.
    '''
    for col in col_list:
        # Get quartiles
        q1, q3 = df[col].quantile([.25, .75])
        
        # Get interquartile range
        iqr = q3 - q1
        
        upper_bound = q3 + k * iqr
        lower_bound = q1 - k * iqr
        
        # Return the dataframe without the outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
        return df

In [ ]:
df = remove_outliers(df, 1.5, df.columns)
df

In [ ]:
# Get the value_counts
cols = df.columns
for col in cols:
    print(col.upper())
    #print(df.col.value_counts())   For some reason this also gives an error
    print(df[col].value_counts())
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print()

In [ ]:
# fips appears to be a categorical number (i.e. like a zip code)
# year_built may also be categorical if we aren't looking at age
df.fips = df.fips.astype(object)
df.year_built = df.year_built.astype(object)
df.info()

# Do my columns leak data from the target column? <br>
- Taxamount can't be calculated unless we have knowledge of tax value <br>
- Taxamount is a function of tax value <br>
- I am dropping tax_amount from the dataframe

In [ ]:
df = df.drop(columns='tax_amount')
df.head()

# Split the data

In [ ]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(df, test_size=.3, random_state=123)
train.shape, validate.shape, test.shape

# Impute year_built with mode

In [ ]:
# Need to fill nulls for: year_built, area_sqft, tax_value

# Create -> Fit -> Use
# Create the object
imputer_year = SimpleImputer(strategy='most_frequent')

# Fit the object to train
imputer_year.fit(train[['year_built']])

# Use the object to impute on train, validate, and test subsets
train[['year_built']]=imputer_year.transform(train[['year_built']])
validate[['year_built']]=imputer_year.transform(validate[['year_built']])
test[['year_built']]=imputer_year.transform(test[['year_built']])

# Now repeat the imputing process for area_sqft
imputer_area = SimpleImputer(strategy='median')
imputer_area.fit(train[['area_sqft']])

train[['area_sqft']]=imputer_area.transform(train[['area_sqft']])
validate[['area_sqft']]=imputer_area.transform(validate[['area_sqft']])
test[['area_sqft']]=imputer_area.transform(test[['area_sqft']])

# Now repeat the imputing process for tax_value
imputer_tax = SimpleImputer(strategy='median')
imputer_tax.fit(train[['tax_value']])

train[['tax_value']]=imputer_tax.transform(train[['tax_value']])
validate[['tax_value']]=imputer_tax.transform(validate[['tax_value']])
test[['tax_value']]=imputer_tax.transform(test[['tax_value']])

In [ ]:
# Verify that all null values have been filled
train.isnull().sum()

In [ ]:
train.head()